In [1]:
(* Required definitions *)
let asrt = function
 | (true,_) -> ()
 | (false, str) -> failwith ("Assertion failure: "^str)

val asrt : bool * string -> unit = <fun>


In [2]:
module Util = struct
  type 'a t = float array array

  let shape (a: 'a t) = 
    let rows = Array.length a in
    let cols = Array.length a.(0) in
    (rows, cols)

  let float_1d size = Array.init size (fun _ -> (Random.float 1.0) -. 0.5)

  let float_2d rows cols = Array.init rows (fun _ -> Array.init cols (fun _ -> (Random.float 1.0) -. 0.5))

  let float_2d_zeros rows cols = Array.make_matrix rows cols 0.0

  let transpose (a : 'a array array) : 'a array array =
    let rows = Array.length a in
    let cols = Array.length a.(0) in
    let transposed = Array.make_matrix cols rows a.(0).(0) in
      for i = 0 to rows - 1 do
        for j = 0 to cols - 1 do
          transposed.(j).(i) <- a.(i).(j)
        done
      done;
    transposed

  let dot_product (a1 : float array array) (a2 : float array array) : float array array =
    let rows_a1, cols_a1 = shape a1 in
    let rows_a2, cols_a2 = shape a2 in
    let transposed = transpose a2 in
    let result = Array.make_matrix rows_a1 cols_a2 0.0 in
      for i = 0 to rows_a1 - 1 do
        for j = 0 to cols_a2 - 1 do
          let dot_product = Array.mapi (fun k x -> x *. transposed.(j).(k)) a1.(i) |> Array.fold_left (+.) 0.0 in
          result.(i).(j) <- dot_product
        done
      done;
    result

  let add_scalar (s : float) (a : float array array) = Array.map (fun row -> Array.map (fun x -> x +. s) row) a
  
  let sub_scalar (a: float array array) (s: float) = Array.map (fun row -> Array.map (fun x -> x -. s) row) a

  let mult_scalar (s : float) (a : float array array) = Array.map (fun row -> Array.map (fun x -> x *. s) row) a
  
  let add_mat (m1 : float array array) (m2 : float array array) : float array array =
    let n_rows1, n_cols1 = shape m1 in
    let n_rows2, n_cols2 = shape m2 in
    if n_rows1 <> n_rows2 || n_cols1 <> n_cols2 then
    let m1_resized = Array.init n_rows1 (fun i -> Array.make n_cols2 m1.(i).(0)) in
    for i = 0 to n_rows1 - 1 do
      for j = 0 to n_cols2 - 1 do
        m1_resized.(i).(j) <- m1.(i).(0)
      done
    done;
    Array.map2 (Array.map2 (+.)) m1_resized m2
    else
    Array.map2 (Array.map2 (+.)) m1 m2

  let sub_mat (l1: float array array) (l2: float array array) = 
    Array.map2 (fun row1 row2 -> Array.map2 (fun x y -> x -. y) row1 row2) l1 l2

  let mult_mat (l1: float array array) (l2: float array array) = 
    Array.map2 (fun row1 row2 -> Array.map2 (fun x y -> x *. y) row1 row2) l1 l2
 
  let sum_2d_matrix (matrix : float array array) : float =
    Array.fold_left (fun acc row -> acc +. Array.fold_left (+.) 0.0 row) 0.0 matrix

  let one_hot (y: int array): float array array =
    let n = Array.fold_left max 0 y + 1 in
    let temp = Array.init (Array.length y) (fun i -> Array.init n (fun j -> if j = y.(i) then 1.0 else 0.0)) in
    transpose temp
    
  let init_params =
    let w1 = float_2d 20 784 in
    let b1 = float_2d 20 1 in
    let v_w1 = float_2d_zeros 20 784 in
    let v_b1 = 0.0 in
    let w2 = float_2d 10 20 in
    let b2 = float_2d 10 1 in
    let v_w2 = float_2d_zeros 10 20 in
    let v_b2 = 0.0 in
    (w1, b1, w2, b2, v_w1, v_b1, v_w2, v_b2)
    
  let random_indices n x y =
  let rec aux acc n =
    if n = 0 then acc
    else
      let r = Random.int (y - x + 1) + x in
      if Array.mem r acc then aux acc n
      else aux (Array.append [|r|] acc) (n-1)
  in aux [||] n

  let select_indices matrix indices =
    Array.map (fun i -> matrix.(i)) indices
end

module Util :
  sig
    type 'a t = float array array
    val shape : 'a t -> int * int
    val float_1d : int -> float array
    val float_2d : int -> int -> float array array
    val float_2d_zeros : int -> int -> float array array
    val transpose : 'a array array -> 'a array array
    val dot_product :
      float array array -> float array array -> float array array
    val add_scalar : float -> float array array -> float array array
    val sub_scalar : float array array -> float -> float array array
    val mult_scalar : float -> float array array -> float array array
    val add_mat : float array array -> float array array -> float array array
    val sub_mat : float array array -> float array array -> float array array
    val mult_mat :
      float array array -> float array array -> float array array
    val sum_2d_matrix : float array array -> float
    val one_hot : int array -> float array array
    val init_params :
      float array array * float array array * float ar

In [3]:
module Activation = struct
  type 'a t = float array array
  type type_ = TanH | ReLU | Sigmoid
  
  let sech x = 1.0 /. cosh x
  let tanh_derivative x = sech x ** 2.0
  
  let relu (z : 'a t) : 'a t =
    Array.map (fun arr -> Array.map (fun x -> max x 0.0) arr) z

  let relu_derive (z : 'a t) : 'a t =
    Array.map (fun arr -> Array.map (fun x -> if x > 0.0 then 1.0 else 0.0) arr) z

  let softmax (z : 'a t) : 'a t =
    let softmax_row row =
      let exp_row = Array.map exp row in
      let sum_exp_row = Array.fold_left (+.) 0.0 exp_row in
      Array.map (fun x -> x /. sum_exp_row) exp_row
    in
    let smax = Array.map softmax_row (Util.transpose z) in
    Util.transpose smax

  let tanh (z : 'a t) : 'a t =
    Array.map (fun arr -> Array.map (fun x -> tanh x) arr) z
    
  let tanh_derive (z : 'a t) : 'a t =
    Array.map (fun arr -> Array.map (fun x -> tanh_derivative x) arr) z

  let sigmoid (z : 'a t) : 'a t =
    let sigmoid_scalar y = 1.0 /. (1.0 +. exp (-. y)) in
    let sigmoid_row row = Array.map sigmoid_scalar row in
    Array.map sigmoid_row z

  let sigmoid_derive (z : 'a t) : 'a t =
    let sigmoid_scalar x = 1.0 /. (1.0 +. exp (-. x)) in
    Array.map (fun row -> Array.map (fun x -> let s = sigmoid_scalar x in s *. (1.0 -. s)) row) z
    
  let activate (z : 'a t) (act : type_) : 'a t = 
    match act with
    | TanH -> tanh z
    | ReLU -> relu z
    | Sigmoid -> sigmoid z
    
  let activate_derive (z : 'a t) (act : type_) : 'a t = 
    match act with
    | TanH -> tanh_derive z
    | ReLU -> relu_derive z
    | Sigmoid -> sigmoid_derive z
    
end

module Activation :
  sig
    type 'a t = float array array
    type type_ = TanH | ReLU | Sigmoid
    val sech : float -> float
    val tanh_derivative : float -> float
    val relu : 'a t -> 'a t
    val relu_derive : 'a t -> 'a t
    val softmax : 'a t -> 'a t
    val tanh : 'a t -> 'a t
    val tanh_derive : 'a t -> 'a t
    val sigmoid : 'a t -> 'a t
    val sigmoid_derive : 'a t -> 'a t
    val activate : 'a t -> type_ -> 'a t
    val activate_derive : 'a t -> type_ -> 'a t
  end


In [4]:
#require "Csv"

let load_csv (file_name: string) (f: string -> 'a) = 
  Csv.load file_name 
  |> List.map Array.of_list
  |> Array.of_list
  |> Array.map (Array.map f)

val load_csv : string -> (string -> 'a) -> 'a array array = <fun>


In [5]:
let train_x : float array array = 
  let temp = load_csv "../data/train_small.csv" float_of_string in
  Util.transpose temp 
  |> Array.map (Array.map (fun x -> x /. 255.0))
  
let train_y : int array = 
  let temp = load_csv "../data/train_small_labels.csv" int_of_string in
  Array.map (fun row -> row.(0)) temp

val train_x : float array array =
  [|[|0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.;
      0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.;
      0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.;
      0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.;
      0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.;
      0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.;
      0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.;
      0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.;
      0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.;
      0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.;
      0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.;
      0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.;
      0.; 0.; 0.; 0.; 0.; 0.; 

val train_y : int array =
  [|1; 0; 1; 4; 0; 0; 7; 3; 5; 3; 8; 9; 1; 3; 3; 1; 2; 0; 7; 5; 8; 6; 2; 0;
    2; 3; 6; 9; 9; 7; 8; 9; 4; 9; 2; 1; 3; 1; 1; 4; 9; 1; 4; 4; 2; 6; 3; 7;
    7; 4; 7; 5; 1; 9; 0; 2; 2; 3; 9; 1; 1; 1; 5; 0; 6; 3; 4; 8; 1; 0; 3; 9;
    6; 2; 6; 4; 7; 1; 4; 1; 5; 4; 8; 9; 2; 9; 9; 8; 9; 6; 3; 6; 4; 6; 2; 9;
    1; 2; 0; 5; 9; 2; 7; 7; 2; 8; 8; 5; 0; 6; 0; 0; 2; 9; 0; 4; 7; 7; 1; 5;
    7; 9; 4; 6; 1; 5; 7; 6; 5; 0; 4; 8; 7; 6; 1; 8; 7; 3; 7; 3; 1; 0; 3; 4;
    5; 4; 0; 5; 4; 0; 3; 5; 1; 0; 8; 3; 7; 0; 9; 6; 6; 9; 5; 4; 6; 9; 3; 5;
    4; 2; 4; 8; 7; 7; 5; 8; 8; 8; 2; 6; 9; 3; 1; 0; 4; 1; 5; 9; 0; 6; 2; 1;
    3; 0; 6; 0; 0; 8; 3; 2; 0; 0; 6; 0; 0; 4; 7; 2; 7; 1; 9; 9; 3; 9; 8; 4;
    6; 6; 5; 3; 8; 1; 8; 7; 1; 3; 7; 6; 3; 6; 3; 6; 3; 2; 3; 2; 2; 7; 9; 2;
    3; 2; 7; 5; 5; 8; 8; 2; 0; 1; 4; 0; 6; 3; 7; 1; 1; 1; 4; 7; 0; 2; 9; 2;
    0; 5; 6; 0; 8; 9; 6; 2; 0; 0; 7; 2; 0; 4; 2; 0; 9; 1; 6; 9; 3; 0; 0; 2;
    0; 6; 8; 4; 0; 7; 2; 1; 9; 5; 2; ...|]


In [6]:
module Prop  = struct
  let forward_prop (w1: float array array) (b1: float array array) (w2: float array array) (b2: float array array) (train_x: float array array) (activate: Activation.type_) = 
    let z1_dot = Util.dot_product w1 train_x in
(*     let _ = asrt(10=100, ""^(string_of_int (fst (Util.shape b1)))^" "^(string_of_int (snd (Util.shape b1)))) in *)
    let z1 = Util.add_mat b1 z1_dot in
    let a1 = Activation.activate z1 activate in
    let z2 = Util.dot_product w2 a1
    |> Util.add_mat b2 in
    let a2 = Activation.sigmoid z2 in
    (z1, a1, a2)
    
  let backward_prop z1 a1 a2 w2 train_x train_y activate:
    (float array array * float * float array array * float) =
    let m_inv = (1. /. (float_of_int (Array.length train_y))) in
    let dz2 = Util.sub_mat a2 train_y in
    let dw2 = Util.dot_product dz2 (Util.transpose a1) 
    |> Util.mult_scalar m_inv in
    let db2 = m_inv *. Util.sum_2d_matrix dz2 in
    let dz1 = Util.dot_product (Util.transpose w2) dz2 
    |> Util.mult_mat (Activation.activate_derive z1 activate) in
    let dw1 = Util.dot_product dz1 (Util.transpose train_x) 
    |> Util.mult_scalar m_inv in
    let db1 = m_inv *. Util.sum_2d_matrix dz1 in
    (dw1, db1, dw2, db2)
    
  let update_params_gd w1 b1 w2 b2 dw1 db1 dw2 db2 lr = 
    let w1 = Util.mult_scalar lr dw1 
    |> Util.sub_mat w1 in
    let b1 = lr *. db1 
    |> Util.sub_scalar b1 in
    let w2 = Util.mult_scalar lr dw2 
    |> Util.sub_mat w2 in
    let b2 = lr *. db2 
    |> Util.sub_scalar b2 in
    (w1, b1, w2, b2)
    
  let update_params_gdm w1 b1 w2 b2 dw1 db1 dw2 db2 v_dw1 v_db1 v_dw2 v_db2 lr beta1 beta2 = 
    let v_dw1 = Util.add_mat (Util.mult_scalar beta1 v_dw1) (Util.mult_scalar beta2 dw1) in
    let v_db1 = (beta2 *. db1) +. (beta1 *. v_db1) in
    let v_dw2 = Util.add_mat (Util.mult_scalar beta1 v_dw2) (Util.mult_scalar beta2 dw2) in
    let v_db2 = (beta2 *. db2) +. (beta1 *. v_db2) in
    let w1 = Util.sub_mat w1 (Util.mult_scalar lr v_dw1) in
    let b1 = Util.sub_scalar b1 (lr *. v_db1) in
    let w2 = Util.sub_mat w2 (Util.mult_scalar lr v_dw2) in
    let b2 = Util.sub_scalar b2 (lr *. v_db2) in
    (w1, b1, w2, b2, v_dw1, v_db1, v_dw2, v_db2)
end

module Prop :
  sig
    val forward_prop :
      float array array ->
      float array array ->
      float array array ->
      float array array ->
      float array array ->
      Activation.type_ ->
      float array array * float array array * float array array
    val backward_prop :
      float array array ->
      float array array ->
      float array array ->
      float array array ->
      float array array ->
      float array array ->
      Activation.type_ ->
      float array array * float * float array array * float
    val update_params_gd :
      float array array ->
      float array array ->
      float array array ->
      float array array ->
      float array array ->
      float ->
      float array array ->
      float ->
      float ->
      float array array * float array array * float array array *
      float array array
    val update_params_gdm :
      float array array ->
      float array array ->
      float array array ->
      float array array ->


In [7]:
module type MONAD = sig
  type 'a t
  val return : 'a -> 'a t
  val (>>=)  : 'a t -> ('a -> 'b t) -> 'b t
  val run : 'a t -> 'a
end

module Monad : MONAD = struct
  type 'a t = 'a
  let return x = x
  let (>>=) x f = f x
  let run x = x
end

module type MONAD =
  sig
    type 'a t
    val return : 'a -> 'a t
    val ( >>= ) : 'a t -> ('a -> 'b t) -> 'b t
    val run : 'a t -> 'a
  end


module Monad : MONAD


In [8]:
module Gd = struct
  type algorithm_type = GD | SGD | MBGD
  let data x y type_ = 
    match type_ with
    | GD -> (x, y)
    | SGD -> 
      let size = snd (Util.shape x) in
      let _ = asrt (snd (Util.shape x) = snd (Util.shape y), "Shape error..") in
      let ri = Util.random_indices 1 0 (size-1) in
      let x', y' = Util.select_indices (Util.transpose x) ri, Util.select_indices (Util.transpose y) ri in
      (Util.transpose x', Util.transpose y')
    | MBGD ->
      let size = snd (Util.shape x) in
      let _ = asrt (snd (Util.shape x) = snd (Util.shape y), "Shape error..") in
      let ri = Util.random_indices 4 0 (size-1) in
      let x', y' = Util.select_indices (Util.transpose x) ri, Util.select_indices (Util.transpose y) ri in
      (Util.transpose x', Util.transpose y')
end

module Gd :
  sig
    type algorithm_type = GD | SGD | MBGD
    val data :
      float array array ->
      float array array ->
      algorithm_type -> float array array * float array array
  end


In [9]:
type optim = VGD | GDM

type optim = VGD | GDM


In [10]:
let gradient_descent train_x train_y lr iterations type_ optim_ beta1 beta2 activate =
  let open Monad in
  let w1, b1, w2, b2, v_w1, v_b1, v_w2, v_b2 = Util.init_params in
  let rec batch_loop i w1 b1 w2 b2 v_w1 (v_b1: float) v_w2 (v_b2: float) iterations train_x train_y (x, y) lr =
    if i = iterations then return (w1, b1, w2, b2)
    else Prop.forward_prop w1 b1 w2 b2 x activate
         |> return >>= fun (z1, a1, a2) ->
         Prop.backward_prop z1 a1 a2 w2 x y activate
         |> return >>= fun (dw1, db1, dw2, db2) ->
         if optim_ = GDM then 
           Prop.update_params_gdm w1 b1 w2 b2 dw1 db1 dw2 db2 v_w1 v_b1 v_w2 v_b2 lr beta1 beta2
           |> return >>= fun (w1', b1', w2', b2', v_dw1', v_db1', v_dw2', v_db2') ->
           if i mod 10 = 0 then Printf.printf "Iteration: %d\n" i;
           batch_loop (i + 1) w1' b1' w2' b2' v_dw1' v_db1' v_dw2' v_db2' iterations train_x train_y (Gd.data train_x train_y type_) lr
         else 
           Prop.update_params_gd w1 b1 w2 b2 dw1 db1 dw2 db2 lr 
           |> return >>= fun (w1', b1', w2', b2') ->
           batch_loop (i + 1) w1' b1' w2' b2' v_w1 v_b1 v_w2 v_b2 iterations train_x train_y (Gd.data train_x train_y type_) lr
  in
  batch_loop 0 w1 b1 w2 b2 v_w1 v_b1 v_w2 v_b2 iterations train_x train_y (Gd.data train_x train_y type_) lr

val gradient_descent :
  float array array ->
  float array array ->
  float ->
  int ->
  Gd.algorithm_type ->
  optim ->
  float ->
  float ->
  Activation.type_ ->
  (float array array * float array array * float array array *
   float array array)
  Monad.t = <fun>


In [11]:
(Util.shape train_x);; 
(Util.shape (Util.one_hot train_y));;

- : int * int = (784, 4000)


- : int * int = (10, 4000)


In [12]:
let result = gradient_descent train_x (Util.one_hot train_y) 0.09 500 MBGD VGD 0.9 0.1 Sigmoid;;
let w1, b1, w2, b2 = Monad.run result;;

val result :
  (float array array * float array array * float array array *
   float array array)
  Monad.t = <abstr>


val w1 : float array array =
  [|[|0.426038888360971146; -0.158361411401767904; 0.0978229458531059137;
      0.366881591014789565; 0.286668034537743921; 0.125315015859070122;
      -0.409011015216012297; -0.188990030213231575; 0.059742196386998736;
      -0.34615215119734255; -0.490233784604748313; 0.256637587325766692;
      -0.251656740474212159; 0.152787092477089326; 0.48717731495272254;
      -0.332554522532503172; 0.0574582248835058262; 0.258516786217372951;
      0.18532904647625259; -0.0324781508558906196; -0.171649560924790556;
      0.225258114528568254; 0.260843272481697697; 0.444543207513039773;
      -0.0466829106219362466; 0.0396486806025953; 0.36819664704757471;
      0.0449662633954558411; 0.130257917730493555; -0.42187402863574619;
      -0.375256835869513194; -0.022730071996965362; -0.229233104415120292;
      0.38945947157575922; -0.199121535032691421; -0.217841553708043145;
      0.0565473408327369365; -0.395243482567093907; 0.10405582793297212;
      0.3604980121271

In [13]:
let get_predictions (a2 : float array array) : int array =
  let n_rows = Array.length a2 in
  let result = Array.make n_rows (-1) in
  for i = 0 to n_rows - 1 do
    let row = a2.(i) in
    let n_cols = Array.length row in
    let max_val = ref row.(0) in
    let max_index = ref 0 in
    for j = 1 to n_cols - 1 do
      let x = row.(j) in
      if x > !max_val then begin
        max_val := x;
        max_index := j;
      end
    done;
    result.(i) <- !max_index
  done;
  result

val get_predictions : float array array -> int array = <fun>


In [14]:
let get_accuracy pred y : float =
  let diff = Array.map2 (fun v1 v2 -> if v1 = v2 then 1 else 0) pred y in
  let sum_and_len = Array.fold_left (fun (s,l) x -> s+x, l+1) (0,0) diff in
  float_of_int (fst (sum_and_len)) /. float_of_int (snd (sum_and_len))

val get_accuracy : 'a array -> 'a array -> float = <fun>


In [15]:
let test_x : float array array = 
  let temp = load_csv "../data/test_small.csv" float_of_string in
  Util.transpose temp 
  |> Array.map (Array.map (fun x -> x /. 255.0))
  
let test_y : int array = 
  let temp = load_csv "../data/test_small_labels.csv" int_of_string in
  Array.map (fun row -> row.(0)) temp

val test_x : float array array =
  [|[|0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.;
      0.|];
    [|0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.;
      0.|];
    [|0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.;
      0.|];
    [|0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.;
      0.|];
    [|0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.;
      0.|];
    [|0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.;
      0.|];
    [|0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.;
      0.|];
    [|0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.;
      0.|];
    [|0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.;
      0.|];
    [|0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.;
      0.|];
    [|0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0.; 0

val test_y : int array =
  [|1; 0; 1; 4; 0; 0; 7; 3; 5; 3; 8; 9; 1; 3; 3; 1; 2; 0; 7|]


In [16]:
Array.length test_y

- : int = 19


In [17]:
let _, _, a2 = Prop.forward_prop w1 b1 w2 b2 test_x Sigmoid in
let preds = get_predictions (Util.transpose a2) in
(preds, get_accuracy preds test_y)

- : int array * float =
([|1; 0; 1; 2; 0; 8; 7; 8; 8; 1; 8; 7; 1; 8; 8; 1; 2; 0; 7|],
 0.578947368421052655)
